In [17]:
import pandas as pd
import openpyxl
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import *
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import time
from math import sqrt

In [18]:
start_time = time.time()
file_path = "F:\\TX DATA.xlsx"
SN = 'Sheet1'
df =  pd.read_excel(file_path,sheet_name = SN, engine= 'openpyxl')

In [19]:
df["Year"] = df["UTC time"].dt.year
df["Month"] =df["UTC time"].dt.month
df['Day'] = df["UTC time"].dt.day
df['Hour'] = df["UTC time"].dt.hour

In [4]:
X = df[['Year', 'Month', 'Day', 'Hour']]
Y = df['D']

In [5]:
model = LinearRegression()
model.fit(X,Y)

LinearRegression()

In [6]:
y_pred = model.predict(X)
print(y_pred)

[37471.48647295 37672.55743215 37873.62839135 ... 50216.24565043
 50417.31660963 50618.38756882]


In [7]:
df['Predictions'] = y_pred
df.to_excel(file_path, sheet_name = SN , index=False)

In [8]:
mse = mean_squared_error(Y, y_pred)
r2 = r2_score(Y, y_pred)
rmse = sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")
print('Mean Squared Error value is: ',mse)
print('R-sqaured value is:',r2)

Root Mean Squared Error: 9872.67343834914
Mean Squared Error value is:  97469680.82028463
R-sqaured value is: 0.12941535190247455


In [9]:
df['Predicted Demand'] = y_pred

In [10]:
df_vif = add_constant(df[['Month', 'Day', 'Hour']])  # adding a constant column for the intercept
vif_data = pd.DataFrame()
vif_data["feature"] = df_vif.columns
vif_data["VIF"] = [variance_inflation_factor(df_vif.values, i) for i in range(df_vif.shape[1])]

In [11]:
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    vif_data.to_excel(writer, sheet_name='VIF Results', index=False)

In [12]:
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
p_values = model.pvalues
print(p_values)

const     0.000000e+00
Year      0.000000e+00
Month    3.417524e-297
Day       9.641417e-01
Hour      0.000000e+00
dtype: float64


In [13]:
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    p_values.to_frame(name='P-Value').to_excel(writer, sheet_name='Regression P-Values', index=True)

In [15]:
book = openpyxl.load_workbook(file_path)
sheet = book['Sheet1']

column = sheet.max_column + 1
sheet.cell(row=1, column=column).value = 'Predict Demand'
for index, value in enumerate(y_pred, start=2):
    sheet.cell(row=index, column=column).value = value
book.save(file_path)

In [16]:
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 150.53902292251587 seconds
